In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt

In [ ]:
# initial actual state
x0 = 0.1


In [ ]:
# Noise covariance in the system
cs_x = 1

In [ ]:
# Noise covariance in the measurement
cs_m = 1

In [ ]:
# duration the chase
iter_num = 75

In [ ]:
# number of particles
N = 20
v = 2

In [ ]:
x_P_update = np.zeros(shape=(N, 1))
z_update = np.zeros(shape=(N, 1))
P_w = np.zeros(shape=(N, 1))
x_p = np.zeros(shape=(N, 1))

In [ ]:
for i in range(N):
    x_p[i] = x0 + np.sqrt(v) + np.random.randn()
z_out = [x0 ** 2 / 20 + np.sqrt(cs_m) * np.random.randn()]

In [ ]:
# the actual output vector for measurement values.
x_out = [x0]  
# time by time output of the particle filters estimate
x_test = x0  
x_test_out = [x_test]

In [ ]:
for t in range(1, iter_num):
    #state update
    x = 0.5 * x0 + 25 * x0 / (1 + x0 ** 2) + 8 * np.cos(1.2 * (t - 1)) + np.sqrt(cs_x) * np.random.randn()
    z = x ** 2 / 20 + np.sqrt(cs_m) * np.random.randn()
    for i in range(1, N):  
        # position update for each particle
        x_P_update[i] = 0.5 * x_p[i] + 25 * x_p[i] / (1 + x_p[i] ** 2) + 8 * math.cos(1.2 * (t - 1)) + np.sqrt(cs_x) * np.random.randn()  
        z_update[i] = x_P_update[i] ** 2 / 20
        # weights assigned for each particle according to z_update and z,particles with higher relevance assigned larger weights
        P_w[i] = (1 / np.sqrt(2 * np.pi * cs_m)) * np.exp(-(z - z_update[i]) ** 2 / (2 * cs_m))  
    #normalisation of the weights
    P_w[0] = [2.21646841e-76]
    P_w = np.divide(P_w, np.sum(P_w)) 
    mar = np.zeros(shape=(1, 10))
    for i in range(1, N):
        mar = ((np.random.rand() <= np.cumsum(P_w)))  ## code for resampling optimisation
        for io in range(1, 10):
            if mar[io] == True:
                x_p[i] = x_P_update[io]
                break
    x_test = np.mean(x_p)
    x_out.append(x)
    z_out.append(z)
    x_test_out.append(x_test)

In [ ]:
# root mean squared error between the estimated and actual positions or states
rmse = np.sqrt(np.mean((x_out - x_test) ** 2))
print(rmse) 

In [ ]:
# plotting out the tracking
x1 = np.arange(0, iter_num)
plt.plot(x1, x_test_out, label="(x_test_out) filtered observation")  
plt.plot(x1, x_out, label="(x_out)  observation")
plt.legend()
plt.title('Observation and Filtered Observation')
plt.show()